### Proyecto Final - Ingenierìa de Datos

Para el proyecto se usarà informaciòn de la tienda ficticia Superstore, una cadena estadounidense que se encuentra en varios estados que ofrece una gran cantidad de productos. la tienda quiere empezar a utilizar su informaciòn para la toma de decisiones, y para esto requiere que se estructure de forma tal que permita hacerlo. Como cualquier negocio nuevo en esto, no saben exactamente por donde empezar, por lo que han contratado nuestro servicio para ayudarlos en entender lo que se tiene y como se puede utilizar.

Luego de algunas reuniones se logra entender la data actual del negocio, los datos actualmente se encuentran en archivos csv separados por varias categorìas, como productos, clientes, ubicaciones, entre otros. Adicional cuentan con una tabla que tiene los registros de todas las compras. Estos archivos de dimensiones son actualizados eventualmente, pero sufren pocos cambios. El que tiene nuevos registros con màs constancia es el de clienetes. El documento con la informaciòn de las transacciones sì tiene nueva informaciòn todos los dìas.


Evaluando varias alternativas, se decide que se utilizaràn los servicios que se encuentran disponibles en Amazon Web Services, ya que permite almacenar archivos planos, creaciòn de data bases y data warehouses y podemos conectar todo usando scripts de python.  

El proyecto con la tienda Superstore, se basarà en subir todos los archivos a un bucket de S3, para posteriormente guardarlos en una base de datos de RDS, ya que estos estàn en primer o segunda forma normal. Luego, se haràn las transformaciones necesarias para unificar las dimensiones y se cargarà todo a un data warehouse, desde donde podremos conectar nuestra herramienta de inteligencia de negocio para empezar a responder preguntas sobre el comportamiento de nuestra informaciòn.

El negocio principalmente quiere saber lo siguiente:

1. En dònde en el paìs se genera la mayor ganancia, para enfocarse en no descuidar esas àreas
2. Què segmento de cliente es el que menos compra, para pensar en una campaña enfocada en ganar ese mercado
3. En què meses del año la demanda es mayor, para anticipar el tema de inventarios
4. Los descuentos que se dan segùn el tipo de envio realizado, para no restar margen a los envìos que màs cuestan a la empresa
5. Evaluar si los productos que màs se venden, son los que màs ganancias generan